In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score

## Features

In [2]:
def calculate_technical_indicators(df, close_col='close', high_col='high', low_col='low', volume_col='volume'):
    """
    Calculate technical indicators for stock price analysis.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with columns for close, high, low, and volume
    close_col (str): Name of closing price column
    high_col (str): Name of high price column
    low_col (str): Name of low price column
    volume_col (str): Name of volume column
    
    Returns:
    pandas.DataFrame: Original data with additional technical indicators
    """
    df = df.copy()
    
    # Trend Indicators
    # Moving Averages
    df['sma_5'] = df[close_col].rolling(window=5).mean()
    df['sma_20'] = df[close_col].rolling(window=20).mean()
    df['sma_50'] = df[close_col].rolling(window=50).mean()
    
    # Exponential Moving Average
    df['ema_12'] = df[close_col].ewm(span=12, adjust=False).mean()
    df['ema_26'] = df[close_col].ewm(span=26, adjust=False).mean()
    
    # MACD
    df['macd'] = df['ema_12'] - df['ema_26']
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # Momentum Indicators
    # Relative Strength Index (RSI)
    delta = df[close_col].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # Stochastic Oscillator
    lookback = 14
    df['lowest_low'] = df[low_col].rolling(window=lookback).min()
    df['highest_high'] = df[high_col].rolling(window=lookback).max()
    df['stoch_k'] = 100 * (df[close_col] - df['lowest_low']) / (df['highest_high'] - df['lowest_low'])
    df['stoch_d'] = df['stoch_k'].rolling(window=3).mean()
    
    # Volatility Indicators
    # Bollinger Bands
    df['bb_middle'] = df[close_col].rolling(window=20).mean()
    df['bb_upper'] = df['bb_middle'] + 2 * df[close_col].rolling(window=20).std()
    df['bb_lower'] = df['bb_middle'] - 2 * df[close_col].rolling(window=20).std()
    
    # Average True Range (ATR)
    high_low = df[high_col] - df[low_col]
    high_close = np.abs(df[high_col] - df[close_col].shift())
    low_close = np.abs(df[low_col] - df[close_col].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    df['atr'] = true_range.rolling(14).mean()
    
    # Volume-based Indicators
    # On-Balance Volume (OBV)
    df['daily_ret'] = df[close_col].pct_change()
    df['obv'] = np.where(df['daily_ret'] > 0, df[volume_col], 
                        np.where(df['daily_ret'] < 0, -df[volume_col], 0)).cumsum()
    
    # Volume-Weighted Average Price (VWAP)
    df['vwap'] = (df[close_col] * df[volume_col]).cumsum() / df[volume_col].cumsum()
    
    # Price Rate of Change
    df['roc_5'] = df[close_col].pct_change(periods=5) * 100
    df['roc_20'] = df[close_col].pct_change(periods=20) * 100
    
    # Additional Derived Features
    df['price_volatility'] = df[close_col].rolling(window=20).std()
    df['volume_volatility'] = df[volume_col].rolling(window=20).std()
    
    return df

## Join with features and sentiment

In [59]:
# apple_df_features = pd.read_csv('price/raw_with_features/AAPL.csv')
# apple_df_prophet = pd.read_csv('price/raw_with_prophet/AAPL_prophet_predictions.csv')
# apple_df_llama_sentiment = pd.read_csv('sentiments/AAPL_sentiment.csv')
# apple_df_llama_sentiment.rename(columns={"date_of_tweets": "Date"}, inplace=True)

# gemini_sentiments = pd.read_csv('sentiments/gemini_sentiment_predictions_all.csv')

# apple_df_gemini_sentiment = gemini_sentiments[gemini_sentiments['ticker'] == 'AAPL']
# apple_df_gemini_sentiment.rename(columns={"date_of_tweets": "Date"}, inplace=True)

# apple_df_combined = apple_df_features.merge(apple_df_prophet[['Date', 'prophet_predicted_price']], on='Date', how='inner')
# apple_df_combined = apple_df_combined.merge(apple_df_llama_sentiment[['Date', 'prediction', 'confidence']].rename(
#         columns={'prediction': 'llama_sentiment', 'confidence': 'llama_sentiment_confidence'}
#     ), on='Date', how='inner')
# apple_df_combined = apple_df_combined.merge(apple_df_gemini_sentiment[['Date', 'prediction', 'confidence']].rename(
#         columns={'prediction': 'gemini_sentiment', 'confidence': 'gemini_sentiment_confidence'}
#     ), on='Date', how='inner')


/var/folders/65/n8y22d1s4nj247nsd99lp98c0000gn/T/ipykernel_86054/1733198241.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apple_df_gemini_sentiment.rename(columns={"date_of_tweets": "Date"}, inplace=True)


In [3]:
def create_df_with_features(ticker: str):
    """
    Loads and combines data for a given ticker from various sources.

    Parameters:
    ticker (str): The stock ticker (e.g., "AAPL").

    Returns:
    pd.DataFrame: A combined DataFrame containing features, predictions, and sentiments.
    """
    try:
        # Define file paths
        features_file = f'price/raw_with_features/{ticker}.csv'
        prophet_file = f'price/raw_with_prophet/{ticker}_prophet_predictions.csv'
        llama_sentiment_file = f'sentiments/{ticker}_sentiment.csv'
        gemini_sentiment_file = 'sentiments/gemini_sentiment_predictions_all.csv'

        # Check file existence
        for file_path in [features_file, prophet_file, llama_sentiment_file, gemini_sentiment_file]:
            if not os.path.exists(file_path):
                raise FileNotFoundError(f"File not found: {file_path}")

        # Load data
        features_df = pd.read_csv(features_file)
        prophet_df = pd.read_csv(prophet_file)
        llama_sentiment_df = pd.read_csv(llama_sentiment_file)
        gemini_sentiments_df = pd.read_csv(gemini_sentiment_file)

        # Standardize column names for consistency
        llama_sentiment_df.rename(columns={"date_of_tweets": "Date"}, inplace=True)
        gemini_sentiment_df = gemini_sentiments_df[gemini_sentiments_df['ticker'] == ticker]
        gemini_sentiment_df.rename(columns={"date_of_tweets": "Date"}, inplace=True)

        # Merge datasets
        combined_df = features_df.merge(
            prophet_df[['Date', 'prophet_predicted_price']], on='Date', how='inner'
        )
        combined_df = combined_df.merge(
            llama_sentiment_df[['Date', 'prediction', 'confidence']].rename(
                columns={'prediction': 'llama_sentiment', 'confidence': 'llama_sentiment_confidence'}
            ), on='Date', how='inner'
        )
        combined_df = combined_df.merge(
            gemini_sentiment_df[['Date', 'prediction', 'confidence']].rename(
                columns={'prediction': 'gemini_sentiment', 'confidence': 'gemini_sentiment_confidence'}
            ), on='Date', how='inner'
        )

        combined_df['pct_change_adj_close'] = combined_df['Adj Close'].pct_change() * 100 # Percentage change in adjusted price compared to the previous day
        combined_df['price_change_direction'] = combined_df['Adj Close'].diff().apply(lambda x: 1 if x >= 0 else -1) 

        # List of sentiment columns to shift
        sentiment_columns = ['prophet_predicted_price', 'llama_sentiment', 
                            'llama_sentiment_confidence', 'gemini_sentiment', 
                            'gemini_sentiment_confidence']

        # Shift the sentiment columns down by one row
        combined_df[sentiment_columns] = combined_df[sentiment_columns].shift(1)

        combined_df = combined_df.dropna()  # Drop rows with missing values - these values do not have enough historical data to calculate indicators
       
        combined_df = combined_df.iloc[1:]  # Drop the first row since we shifted the sentiment columns

        return combined_df

    except FileNotFoundError as e:
        print(f"Error: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if files are missing
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on any other exception

In [4]:
AAPL_combined_df = create_df_with_features("AAPL")
KO_combined_df = create_df_with_features("KO")
TSLA_combined_df = create_df_with_features("TSLA")
V_combined_df = create_df_with_features("V")
XOM_combined_df = create_df_with_features("XOM")

/var/folders/65/n8y22d1s4nj247nsd99lp98c0000gn/T/ipykernel_23281/3295113978.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gemini_sentiment_df.rename(columns={"date_of_tweets": "Date"}, inplace=True)
/var/folders/65/n8y22d1s4nj247nsd99lp98c0000gn/T/ipykernel_23281/3295113978.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gemini_sentiment_df.rename(columns={"date_of_tweets": "Date"}, inplace=True)
/var/folders/65/n8y22d1s4nj247nsd99lp98c0000gn/T/ipykernel_23281/3295113978.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: h

In [5]:
os.makedirs("data", exist_ok=True)

AAPL_combined_df.to_csv("data/AAPL_combined.csv", index=False)
KO_combined_df.to_csv("data/KO_combined.csv", index=False)
TSLA_combined_df.to_csv("data/TSLA_combined.csv", index=False)
V_combined_df.to_csv("data/V_combined.csv", index=False)
XOM_combined_df.to_csv("data/XOM_combined.csv", index=False)

1	0.919434	0	0.999998

1	0.779785	1	0.982422

In [6]:
file_paths = ["data/AAPL_combined.csv", "data/KO_combined.csv", "data/TSLA_combined.csv", "data/V_combined.csv", "data/XOM_combined.csv"]
tickers = ["AAPL", "KO", "TSLA", "V", "XOM"]

# Read and combine data
dataframes = []
for file, ticker in zip(file_paths, tickers):
    df = pd.read_csv(file)
    df['ticker'] = ticker  # Add ticker column
    dataframes.append(df)

# Combine all dataframes
combined_df = pd.concat(dataframes, ignore_index=True)

# One-hot encode the ticker column
combined_df = pd.get_dummies(combined_df, columns=['ticker'], prefix='', prefix_sep='')

# Save combined DataFrame (optional)
combined_df.to_csv("data/combined_data.csv", index=False)

## Train Test Split

In [7]:
from sklearn.model_selection import TimeSeriesSplit

In [10]:
def save_timeseries_splits(data, n_splits=5):
    """
    Performs Time Series Split on the training part of data, reserves last 30% of data as a test set.
    
    Args:
        data: DataFrame with features and target for time-series analysis
        ticker: The stock ticker name to organize data into its own folder
        n_splits: Number of splits for TimeSeriesSplit
    """
    
    # Determine the split for training-validation and test set
    test_size = int(len(data) * 0.3)  # Reserve last 30% for testing
    train_val_data = data.iloc[:-test_size]  # First 70% for training-validation
    test_data = data.iloc[-test_size:]  # Last 30% for testing

    # Set up time-series split
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Directory to save splits
    os.makedirs('data_split', exist_ok=True)

    # Perform splits
    for i, (train_index, val_index) in enumerate(tscv.split(train_val_data), 1):
        # Split the data
        train_df = train_val_data.iloc[train_index]
        val_df = train_val_data.iloc[val_index]

        # Save training and validation split
        split_subdir = os.path.join('data_split', f"split_{i:02d}")
        os.makedirs(split_subdir, exist_ok=True)
        
        train_df.to_csv(os.path.join(split_subdir, 'train.csv'), index=False)
        val_df.to_csv(os.path.join(split_subdir, 'val.csv'), index=False)

        print(f"Saved split {i} at {split_subdir}")
    
    # Save the test set
    test_data_dir = os.path.join('data_split', "test")
    os.makedirs(test_data_dir, exist_ok=True)
    test_data.to_csv(os.path.join(test_data_dir, 'test.csv'), index=False)
    print(f"Saved test set {test_data_dir}")

In [11]:
n_splits = 5

file_path = f"data/combined_data.csv"
if os.path.exists(file_path):
    data = pd.read_csv(file_path)
    save_timeseries_splits(data, n_splits)
else:
    print(f"File {file_path} not found.")

Saved split 1 at data_split/split_01
Saved split 2 at data_split/split_02
Saved split 3 at data_split/split_03
Saved split 4 at data_split/split_04
Saved split 5 at data_split/split_05
Saved test set data_split/test


In [8]:
def create_train_test_split(data):
    # Determine the split for training-validation and test set
    test_size = int(len(data) * 0.3)  # Reserve last 30% for testing
    train_val_data = data.iloc[:-test_size]  # First 70% for training-validation
    test_data = data.iloc[-test_size:]  # Last 30% for testing

    train_val_data_dir = os.path.join('data_split', "train_val")
    os.makedirs(train_val_data_dir, exist_ok=True)
    train_val_data.to_csv(os.path.join(train_val_data_dir, 'train_val.csv'), index=False)
    print(f"Saved train val set {train_val_data_dir}")

    test_data_dir = os.path.join('data_split', "test")
    os.makedirs(test_data_dir, exist_ok=True)
    test_data.to_csv(os.path.join(test_data_dir, 'test.csv'), index=False)
    print(f"Saved test set {test_data_dir}")

In [9]:
file_path = f"data/combined_data.csv"
data = pd.read_csv(file_path)
create_train_test_split(data)

Saved train val set data_split/train_val
Saved test set data_split/test


## Modeling

In [57]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "LightGBM": lgb.LGBMRegressor(random_state=42)
}

param_grids = {
    "Linear Regression": {},
    "Ridge": {
        "alpha": [0.001, 0.01, 0.1, 1, 10, 100, 1000],  # Regularization strength
    },
    "Lasso": {
        "alpha": [0.001, 0.01, 0.1, 1, 10, 100, 1000],  # Regularization strength
        "max_iter": [1000, 5000, 10000],  # Maximum number of iterations
    },
    "Random Forest": {
        "n_estimators": [50, 100, 200, 300, 500],  # Number of trees in the forest
        "max_depth": [None, 5, 10, 20, 30],  # Maximum depth of the tree
        "min_samples_split": [2, 5, 10, 20],  # Minimum samples required to split an internal node
        "min_samples_leaf": [1, 2, 4, 8],  # Minimum samples required to be at a leaf node
    },
    "XGBoost": {
        "n_estimators": [50, 100, 200, 300, 500],  # Number of boosting rounds
        "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],  # Step size at each iteration
        "max_depth": [3, 5, 6, 10],  # Maximum depth of the tree
        "min_child_weight": [1, 2, 5, 10],  # Minimum sum of instance weight (hessian) needed in a child
    },
    "LightGBM": {
        "n_estimators": [50, 100, 200, 300, 500],  # Number of boosting rounds
        "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],  # Step size at each iteration
        "max_depth": [3, 5, 6, 10],  # Maximum depth of the tree
        "num_leaves": [31, 50, 100],  # Maximum number of leaves in one tree
    }
}

# TimeSeriesSplit for automatic splitting
time_series_cv = TimeSeriesSplit(n_splits=5)

In [58]:
def train_and_evaluate_with_sentiment(train_val_df, model_name):
    # Select features and target
    X_columns = train_val_df.columns[train_val_df.columns.get_loc("Volume") + 1:-1]  # Select all columns after 'Volume', remove the last ticker column
    X_columns = X_columns.drop(['price_change_direction', 'pct_change_open'])
    y_column = 'pct_change_open'
    
    X = train_val_df[X_columns].values
    y = train_val_df[y_column].values
    
    # Initialize the model and parameter grid
    model = models[model_name]
    param_grid = param_grids[model_name]
    
    # Set up GridSearchCV with TimeSeriesSplit
    grid_search = GridSearchCV(model, param_grid, cv=time_series_cv, scoring='neg_mean_squared_error')
    
    # Fit the grid search (this will automatically split the data)
    grid_search.fit(X, y)
    
    # Best model and parameters
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    # Predict using the best model found
    predictions = best_model.predict(X)
    
    # Evaluate performance (RMSE)
    rmse = np.sqrt(mean_squared_error(y, predictions))
    
    return rmse, best_params

In [65]:
!ls

README.md              model.ipynb            stocktable.pdf
data                   price                  tweet
data_split             prophet.ipynb
generate_sentiments.py sentiments


In [66]:
train_val_file = '/Users/abhinavkrishnan/Documents/Fall 2024/Capstone/sn2/data_split/train_val/train_val.csv'
train_val_df = pd.read_csv(train_val_file)

model_results = {}
for model_name in models:
    print(model_name)
    rmse, best_params = train_and_evaluate_with_sentiment(df, model_name)
    model_results[model_name] = {
        'RMSE': rmse,
        'Best Params': best_params
    }

Linear Regression
Ridge
Lasso


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.6425e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.91968e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.69914e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.50208e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_ridge.p

Random Forest


KeyboardInterrupt: 

In [ ]:
model_results

In [ ]:
# # Set the directory where your splits are stored
# base_dir = 'data_split/'  # Replace with the actual path to your splits directory

# # Loop through the directory splits
# split_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

# # Function to train and evaluate the Random Forest model on a given split
# # def train_and_evaluate_with_sentiment(train_df, val_df):
# #     # Select features and target
# #     X_columns = train_df.columns[train_df.columns.get_loc("Volume") + 1:-1]  # Select all columns after 'Volume', remove the last ticker column
# #     y_column = "Open"
    
# #     X_train = train_df[X_columns].values
# #     y_train = train_df[y_column].values
# #     X_val = val_df[X_columns].values
# #     y_val = val_df[y_column].values
    
# #     # Initialize the Random Forest model
# #     model = RandomForestRegressor(n_estimators=100, random_state=42)
    
# #     # Train the model
# #     model.fit(X_train, y_train)
    
# #     # Predict on validation data
# #     predictions = model.predict(X_val)
    
# #     # Evaluate performance
# #     rmse = np.sqrt(mean_squared_error(y_val, predictions))
# #     return rmse

# def train_and_evaluate_with_sentiment(train_df, val_df, model_name):
#     # Select features and target
#     X_columns = train_df.columns[train_df.columns.get_loc("Volume") + 1:-1]  # Select all columns after 'Volume', remove the last ticker column
#     X_columns = X_columns.drop(['price_change_direction', 'pct_change_open'])
#     y_column = 'pct_change_open'
    
#     X_train = train_df[X_columns].values
#     y_train = train_df[y_column].values
#     X_val = val_df[X_columns].values
#     y_val = val_df[y_column].values
    
#     # Initialize the Random Forest model
#     model = models[model_name]
#     param_grid = param_grids[model_name]

#     if param_grid:
#         grid_search = GridSearchCV(model, param_grid)
    
#     # Train the model
#     model.fit(X_train, y_train)
    
#     # Predict on validation data
#     predictions = model.predict(X_val)
    
#     # Evaluate performance
#     rmse = np.sqrt(mean_squared_error(y_val, predictions))
#     return rmse

In [ ]:
# # Loop through each split directory, load train/val, and evaluate the model
# for split in split_dirs:
#     train_path = os.path.join(base_dir, split, 'train.csv')  # Path to the training CSV
#     val_path = os.path.join(base_dir, split, 'val.csv')  # Path to the validation CSV
    
#     # Ensure both train and val files exist
#     if os.path.exists(train_path) and os.path.exists(val_path):
#         # Load train and val data
#         train_df = pd.read_csv(train_path)
#         val_df = pd.read_csv(val_path)
        
#         # Train and evaluate the Random Forest model
#         rmse = train_and_evaluate_with_sentiment(train_df, val_df)
#         print(f"Split: {split} -> RMSE: {rmse}")
#     else:
#         print(f"Skipping split {split} because train or val file is missing.")